## Read data from database

In [14]:
import os
# specify which GPU will be used
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [15]:
import os

import pymysql
from warnings import filterwarnings

_connection = None

def get_connection(db_config):
    """
    get db connection
    :return:
    """
    global _connection
    if _connection is None:
        _connection = pymysql.connect(host=db_config['host'], user=db_config['username'],
                                      password=db_config['password'],
                                      db=db_config['db'], charset="utf8")
        filterwarnings('ignore', category=pymysql.Warning)

    return _connection


def close():
    """
    close DB connection
    :return:
    """
    global _connection
    if _connection is not None:
        _connection.close()
    _connection = None

In [16]:
db = {
    'host': '172.26.187.242',
    'username': 'malware_r',
    'password': 'GEg22v2O7jbfWhb3',
    'db': 'malware'
}

### Fields

- mw_file_suffix: file name after hash value
- mw_file_prefix: directory
- mw_em_f: features of ember, splitted by ";"

In [17]:
import time

# the base function which can query sql and return dict data
def get_specific_data(table_suffix, sql=None):
    start_time = time.time()
    
    global _connection
    if _connection is None:
        raise Exception("please init db connect first")

    cursor = _connection.cursor()
    cursor.execute("SET NAMES utf8mb4")

    ret = []
        
    cursor.execute(sql)

    field_names = [i[0] for i in cursor.description]

    for row in cursor:
        temp = {}
        for key in range(len(row)):
            temp[field_names[key]] = row[key]
        ret.append(temp)
     
    cursor.close()
    # _connection.close()
    print("--- %s seconds ---" % (time.time() - start_time))
    
    return ret

In [18]:
close()
res1 = []
get_connection(db)
table_suffix = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F"]
table_suffix = ["0","8","F"]
# Iterate all partitions of databases
for suffix in table_suffix:
    sql = """ 
select
  a.mw_file_hash,
  a.section_name,
  c.mw_file_suffix as mw_file_size,
  c.mw_file_prefix as mw_file_directory,
  c.mw_num_engines,
  a.pointerto_raw_data,
  a.virtual_size,
  d.mw_em_f
from mw_index_2017_section_%s as a
  inner join mw_index_2017_%s c on a.mw_file_hash = c.mw_file_hash
  inner join mw_index_2017_feature_%s d on a.mw_file_hash = d.mw_file_hash
where CNT_CODE = 1 and MEM_EXECUTE = 1 and c.mw_num_engines <> -1 and (c.mw_num_engines >= 4 or c.mw_num_engines = 0) and
      c.mw_file_prefix in ('201702')
    """ % (suffix, suffix, suffix)
    res1.extend(get_specific_data(suffix, sql))
close()
print(len(res1))

--- 5.226050138473511 seconds ---
--- 7.007890701293945 seconds ---
--- 5.362804174423218 seconds ---
58589


In [19]:
close()
res2 = []
get_connection(db)
table_suffix = ["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F"]
table_suffix = ["0","8","F"]
# Iterate all partitions of databases
for suffix in table_suffix:
    sql = """ 
select
  a.mw_file_hash,
  a.section_name,
  c.mw_file_suffix as mw_file_size,
  c.mw_file_prefix as mw_file_directory,
  c.mw_num_engines,
  a.pointerto_raw_data,
  a.virtual_size,
  d.mw_em_f
from mw_index_2017_section_%s as a
  inner join mw_index_2017_%s c on a.mw_file_hash = c.mw_file_hash
  inner join mw_index_2017_feature_%s d on a.mw_file_hash = d.mw_file_hash
where CNT_CODE = 1 and MEM_EXECUTE = 1 and c.mw_num_engines <> -1 and (c.mw_num_engines >= 4 or c.mw_num_engines = 0) and
      c.mw_file_prefix in ('201705')
    """ % (suffix, suffix, suffix)
    res2.extend(get_specific_data(suffix, sql))
close()
print(len(res2))

--- 3.4539613723754883 seconds ---
--- 2.776331901550293 seconds ---
--- 1.7263424396514893 seconds ---
24465


## Check and split data

In [20]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import pylab as pl
from sklearn.model_selection import train_test_split

max_length = 300000

train_data = pd.DataFrame(res1)
# train_data = train_data.loc[train_data.virtual_size <= max_length]
# train_data = train_data.reset_index(drop=True)
train_data.mw_num_engines[train_data.mw_num_engines == 0 ] = 0
train_data.mw_num_engines[train_data.mw_num_engines >= 4 ] = 1
train_label = train_data.mw_num_engines.ravel()

test_data = pd.DataFrame(res2)
# test_data = test_data.loc[test_data.virtual_size <= max_length]
# test_data = test_data.reset_index(drop=True)
test_data.mw_num_engines[test_data.mw_num_engines == 0 ] = 0
test_data.mw_num_engines[test_data.mw_num_engines >= 4 ] = 1
test_label = test_data.mw_num_engines.ravel()

x_train, x_val, y_train, y_val = train_test_split(train_data, train_label, test_size=0.1, random_state=2345)
x_test = test_data
y_test = test_label

x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ip

In [21]:
import pandas as pd
import re
import hashlib
import numpy as np
from sklearn.feature_extraction import FeatureHasher
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss, confusion_matrix

In [22]:
def get_ember_feature(data):
    ember_f = np.zeros((len(data.mw_em_f), 2351), dtype=float)
    for index, item in data.iterrows():
        ember_f[index, :] = item['mw_em_f'].split(';')
    return ember_f

In [23]:
def get_model(x_train, y_train, x_val, y_val):
    params = {'application': 'binary'}
    lgbm_dataset = lgb.Dataset(x_train, y_train.ravel())
    valid_sets = lgb.Dataset(x_val, y_val.ravel())

    model = lgb.train(params, lgbm_dataset, valid_sets=valid_sets, early_stopping_rounds=10)
    y_pred = model.predict(x_val, num_iteration=model.best_iteration)
    
    loss = log_loss(y_val, y_pred)
    auc = roc_auc_score(y_val, y_pred)
    acc = accuracy_score(y_val, (y_pred > 0.5).astype(int))
#     model.save_model(file_path + "-%04d-%.5f-%.5f.h5" % (model.best_iteration, loss, acc),
#                      num_iteration=model.best_iteration)
    print("val loss : %.5f" % loss)
    print("auc score : %.5f" % auc)
    print("accuracy score : %.5f" % acc)
        
    return model

In [24]:
x_etrain = get_ember_feature(x_train)
x_eval = get_ember_feature(x_val)
x_etest = get_ember_feature(x_test)

In [25]:
import matplotlib.pyplot as plt
model = get_model(x_etrain, y_train, x_eval, y_val)

[1]	valid_0's binary_logloss: 0.520668
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.463745
[3]	valid_0's binary_logloss: 0.417665
[4]	valid_0's binary_logloss: 0.379067
[5]	valid_0's binary_logloss: 0.346971
[6]	valid_0's binary_logloss: 0.319679
[7]	valid_0's binary_logloss: 0.29645
[8]	valid_0's binary_logloss: 0.276105
[9]	valid_0's binary_logloss: 0.25832
[10]	valid_0's binary_logloss: 0.2428
[11]	valid_0's binary_logloss: 0.22914
[12]	valid_0's binary_logloss: 0.216891
[13]	valid_0's binary_logloss: 0.206268
[14]	valid_0's binary_logloss: 0.197075
[15]	valid_0's binary_logloss: 0.188283
[16]	valid_0's binary_logloss: 0.180711
[17]	valid_0's binary_logloss: 0.173922
[18]	valid_0's binary_logloss: 0.167557
[19]	valid_0's binary_logloss: 0.161919
[20]	valid_0's binary_logloss: 0.156859
[21]	valid_0's binary_logloss: 0.152155
[22]	valid_0's binary_logloss: 0.147893
[23]	valid_0's binary_logloss: 0.14401
[24]	valid_0's binary_logloss: 0.

In [26]:
def estimate_model(y_pred, test_y):
    
    loss = log_loss(test_y, y_pred)
    auc = roc_auc_score(test_y, y_pred)
    acc = accuracy_score(test_y, (y_pred > 0.5).astype(int))
    print("loss : %.5f" % loss)
    print("auc score : %.5f" % auc)
    print("accuracy score : %.5f" % acc)

    fp_np_index = np.where(test_y == 0)
    fp_np = y_pred[fp_np_index].shape[0]
    thre_index = int(np.ceil(fp_np - fp_np * 0.001))

    sorted_pred_prob = np.sort(y_pred[fp_np_index], axis=0)
    thre = sorted_pred_prob[thre_index]
    if thre == 1:
        thre = max(sorted_pred_prob[np.where(sorted_pred_prob != 1)])

    y_pred_prob = np.vstack((y_pred.transpose(), (1 - y_pred).transpose())).transpose()
    y_pred_prob[:, 1] = thre
    y_pred_label = np.argmin(y_pred_prob, axis=-1)

    tn, fp, fn, tp = confusion_matrix(test_y, y_pred_label).ravel()
    fp_rate = fp / (fp + tn)
    recall_rate = tp / (tp + fn)

    print("thre: %.10f"%  thre)
    print("fp:  %.10f"%  fp_rate)
    print("recall:  %.10f"%  recall_rate)
    
    return auc, loss, recall_rate

In [27]:
y_p = model.predict(x_etest)
y_pred_e = np.zeros((len(y_p), 1))
for i in range(len(y_p)):
    y_pred_e[i, 0] = y_p[i]

estimate_model(y_pred_e, y_test)

loss : 0.10468
auc score : 0.99249
accuracy score : 0.96354
thre: 0.9677302462
fp:  0.0008908686
recall:  0.5603382014


(0.9924880381546489, 0.10467744166450532, 0.5603382013835511)

## Malcon

In [28]:
import keras
import numpy as np

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'

    def __init__(self, list_IDs, datasets, labels, batch_size=32, dim=8192, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.datasets = datasets
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'  # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.zeros((self.batch_size, self.dim), dtype=float)
        y = np.zeros(self.batch_size, dtype=float)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            base_path = "/ssd/2017/{0}/{1}{2}"
            item = self.datasets.loc[ID]
            file_path = base_path.format(item["mw_file_directory"], item["mw_file_hash"], item["mw_file_size"])
            in_file = open(file_path, 'rb')
            in_file.seek(item['pointerto_raw_data'])
            if item['virtual_size'] > max_length:
                bytes_data = [int(single_byte) for single_byte in in_file.read(max_length)]
            else:
                bytes_data = [int(single_byte) for single_byte in in_file.read(item['virtual_size'])]
            X[i, 0:len(bytes_data)] = bytes_data
            y[i] = self.labels[ID]

        return X, y

Using TensorFlow backend.


In [29]:
import hashlib
import json
import time

import keras
from keras import Input
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.layers import Dense, Embedding, Conv1D, Multiply, GlobalMaxPooling1D, Dropout, Activation
from keras.models import load_model
from sklearn.model_selection import train_test_split

class TMalConv(object):
    """
    train of mal conv
    """

    def __init__(self):
        self.max_len = max_length
        self.history = None
        self.model = None
        self.p_md5 = None
        self.time = time.time()
        self.summary = {
            'time':time.time(),
            'batch_size': 32,
            'epochs': 64,
            'g_c_filter': 128,
            'g_c_kernel_size': 500,
            'g_c_stride': 500,
        }

    def run(self):
        """
        :return:
        """
        self.train()
        
    def get_p(self, key):
        """
        get the parameter from the summary
        :param key:
        :return:
        """
        return self.summary[key]

    def gate_cnn(self, gate_cnn_input):
        """
        construct a gated cnn by the specific kernel size
        :param gate_cnn_input:
        :param kernel_size:
        :return:
        """
        gate_cnn_input = Activation('relu')(gate_cnn_input)
        
        conv1_out = Conv1D(self.get_p("g_c_filter"), self.get_p("g_c_kernel_size"), strides=self.get_p("g_c_stride"))(
            gate_cnn_input)
        conv2_out = Conv1D(self.get_p("g_c_filter"), self.get_p("g_c_kernel_size"), strides=self.get_p("g_c_stride"),
                           activation="sigmoid")(gate_cnn_input)
        merged = Multiply()([conv1_out, conv2_out])
        gate_cnn_output = GlobalMaxPooling1D()(merged)
        return gate_cnn_output

    def get_model(self):
        """
        get a model
        :param max_len:
        :param kernel_sizes:
        :return:
        """
        net_input = Input(shape=(self.max_len,))

        embedding_out = Embedding(256, 8, input_length=self.max_len)(net_input)
        merged = self.gate_cnn(embedding_out)

        dense_out = Dense(128)(merged)
        
        net_output = Dense(1, activation='sigmoid')(dense_out)

        model = keras.models.Model(inputs=net_input, outputs=net_output)
        
        model.summary()
        
        return model

    def train(self):
        batch_size = self.get_p("batch_size")
        epochs = self.get_p("epochs")

        self.model = self.get_model()

        print('Length of the train: ', len(x_train))
        print('Length of the validation: ', len(x_val))
        
#         tensor_board = TensorBoard(log_dir='./logs/', batch_size=batch_size)
        file_path = "/home/zhaoqi/BaseTrain/models/"+ str(self.time) +"-{epoch:04d}-{val_loss:.5f}-{val_acc:.5f}.h5"
        early_stopping = EarlyStopping("val_loss", patience=3, verbose=0, mode='auto')
        check_point = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
        callbacks_list = [check_point, early_stopping]

        # Generators
        training_generator = DataGenerator(range(len(x_train)), x_train, y_train, batch_size, self.max_len)
        validation_generator = DataGenerator(range(len(x_val)), x_val, y_val, batch_size, self.max_len)

        self.model.compile(loss='binary_crossentropy',
                           optimizer='adam',
                           metrics=['accuracy'])

        self.model.fit_generator(generator=training_generator,
                                 validation_data=validation_generator,
                                 use_multiprocessing=True,
                                 epochs=epochs,
                                 workers=6,
                                 callbacks=callbacks_list)

In [31]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [32]:
t_instance = TMalConv()
t_instance.run()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 300000)       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 300000, 8)    2048        input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 300000, 8)    0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 600, 128)     512128      activation_1[0][0]               
__________________________________________________________________________________________________
conv1d_2 (

/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.188627). Check your callbacks.
  % delta_t_median)


1647/1647 [==============================] - 936s 568ms/step - loss: 0.2285 - acc: 0.9158 - val_loss: 0.1919 - val_acc: 0.9310

Epoch 00001: val_loss improved from inf to 0.19186, saving model to /home/zhaoqi/BaseTrain/models/1533004410.4653583-0001-0.19186-0.93101.h5
Epoch 2/64
 332/1647 [=====>........................] - ETA: 12:39 - loss: 0.1601 - acc: 0.9490

/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.193544). Check your callbacks.
  % delta_t_median)


1321/1647 [=======================>......] - ETA: 3:10 - loss: 0.1593 - acc: 0.9470

...

1051/1647 [==================>...........] - ETA: 5:57 - loss: 0.0894 - acc: 0.9799

/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.192335). Check your callbacks.
  % delta_t_median)


1647/1647 [==============================] - 1107s 672ms/step - loss: 0.0933 - acc: 0.9784 - val_loss: 0.1866 - val_acc: 0.9421

Epoch 00004: val_loss did not improve from 0.17470
Epoch 5/64
 671/1647 [===========>..................] - ETA: 9:32 - loss: 0.0815 - acc: 0.9828

/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.188902). Check your callbacks.
  % delta_t_median)


1647/1647 [==============================] - 1116s 677ms/step - loss: 0.0852 - acc: 0.9816 - val_loss: 0.2047 - val_acc: 0.9392

Epoch 00005: val_loss did not improve from 0.17470


In [33]:
model_dir = '/home/zhaoqi/BaseTrain/models/'
f_name = '1533004410.4653583-0002-0.17470-0.94040.h5'
c_model = load_model(model_dir + f_name)

test_generator = DataGenerator(range(len(x_test)), x_test, y_test, 32, max_length, False)
y_pred = c_model.predict_generator(generator=test_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

764/764 [==============================] - 412s 539ms/step


In [34]:
estimate_model(y_pred, y_test[0:len(y_pred)])

loss : 0.25448
auc score : 0.94617
accuracy score : 0.89508
thre: 0.9772065878
fp:  0.0008914642
recall:  0.2730769231


(0.9461725969038761, 0.25447557655325487, 0.27307692307692305)

## Merge features

### Merge Malconv and Ember

In [37]:
f_x_test = x_test.iloc[0:len(y_pred)]
f_x_test['pred'] = y_pred
f_x_test = f_x_test[['mw_file_hash','mw_num_engines','pred']]
f_x_test = f_x_test.groupby('mw_file_hash').max().reset_index(drop=True)
estimate_model(f_x_test['pred'].values, f_x_test['mw_num_engines'].values)

loss : 0.25529
auc score : 0.93668
accuracy score : 0.89484
thre: 0.9787095785
fp:  0.0009366219
recall:  0.3027277155


/home/zhaoqi/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(0.9366812637141965, 0.255294795930398, 0.30272771553823674)

In [38]:
from keras.models import Model

model_f = Model(c_model.input, c_model.layers[-3].output)

train_generator = DataGenerator(range(len(x_train)), x_train, y_train, 32, max_length, False)
malcon_train_x = model_f.predict_generator(generator=train_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

val_generator = DataGenerator(range(len(x_val)), x_val, y_val, 32, max_length, False)
malcon_val_x = model_f.predict_generator(generator=val_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

test_generator = DataGenerator(range(len(x_test)), x_test, y_test, 32, max_length, False)
malcon_test_x = model_f.predict_generator(generator=test_generator, max_queue_size=10, workers=6, use_multiprocessing=True, verbose=1)

764/764 [==============================] - 156s 204ms/step


In [50]:
def get_max_feature(origin_data, feature_data):
    feature_data = pd.DataFrame(feature_data)
    feature_data['mw_file_hash'] = origin_data.iloc[0:len(feature_data)][['mw_file_hash']]
    feature_data = feature_data.groupby('mw_file_hash').first().merge(origin_data[['mw_file_hash','mw_em_f','mw_num_engines']], how='inner', on='mw_file_hash')
    tmp_ember = get_ember_feature(feature_data[['mw_em_f']])
    tmp_label = feature_data[['mw_num_engines']].mw_num_engines
    tmp_data = feature_data.drop(['mw_num_engines', 'mw_em_f', 'mw_file_hash'], axis=1)
    return tmp_data, tmp_label, tmp_ember

In [51]:
max_train_data, max_train_label, max_train_ember = get_max_feature(x_train, malcon_train_x)
max_val_data, max_val_label, max_val_ember = get_max_feature(x_val, malcon_val_x)
max_test_data, max_test_label, max_test_ember = get_max_feature(x_test, malcon_test_x)

In [52]:
def merge_feature(m_data, e_data):
    num = len(m_data)
    m_x = np.zeros((num, 128+2351), dtype=float)
    
    for index in range(num):
        m_x[index, 0:128] = m_data.iloc[index]
        m_x[index, 128:128+2351] = e_data[index]  
    return m_x

In [53]:
merge_train_x = merge_feature(max_train_data, max_train_ember)
merge_val_x = merge_feature(max_val_data, max_val_ember)
merge_test_x = merge_feature(max_test_data, max_test_ember)

In [56]:
model_m = get_model(merge_train_x[:, 0:128], max_train_label, merge_val_x[:, 0:128], max_val_label )

[1]	valid_0's binary_logloss: 0.524031
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.46987
[3]	valid_0's binary_logloss: 0.426163
[4]	valid_0's binary_logloss: 0.389946
[5]	valid_0's binary_logloss: 0.360472
[6]	valid_0's binary_logloss: 0.335289
[7]	valid_0's binary_logloss: 0.313761
[8]	valid_0's binary_logloss: 0.295727
[9]	valid_0's binary_logloss: 0.280041
[10]	valid_0's binary_logloss: 0.266274
[11]	valid_0's binary_logloss: 0.254547
[12]	valid_0's binary_logloss: 0.244384
[13]	valid_0's binary_logloss: 0.234946
[14]	valid_0's binary_logloss: 0.227119
[15]	valid_0's binary_logloss: 0.220544
[16]	valid_0's binary_logloss: 0.214707
[17]	valid_0's binary_logloss: 0.209596
[18]	valid_0's binary_logloss: 0.204982
[19]	valid_0's binary_logloss: 0.200993
[20]	valid_0's binary_logloss: 0.19756
[21]	valid_0's binary_logloss: 0.194767
[22]	valid_0's binary_logloss: 0.192512
[23]	valid_0's binary_logloss: 0.190222
[24]	valid_0's binary_logloss

In [57]:
y_p = model_m.predict(merge_test_x[:, 0:])
y_pred = np.zeros((len(y_p), 1))
for i in range(len(y_p)):
    y_pred[i, 0] = y_p[i]

estimate_model(y_pred, max_test_label)

loss : 0.27543
auc score : 0.93688
accuracy score : 0.89555
thre: 0.9819630483
fp:  0.0008914642
recall:  0.0395202214


(0.9368794265841063, 0.2754346038873211, 0.03952022143626019)